Для временного ряда bike-sharing построить три модели:
* Датасет аггрегировать по неделям
* Простое экспоненциальное сглаживание
* Двойное экспоненциальное сглаживание
* Тройное экспоненциальное сглаживание (тип модели additive или multiplicative) выбрать самостоятельно 


Для финального отчета сделать кросс-валидацию, пояснить выбранные метрики, а так же их финальные значения. 





In [47]:
import pandas as pd
from plotly.offline import iplot
from plotly.graph_objects import *

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,9.84,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,9.02,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,9.02,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,9.84,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,9.84,0.2879,0.75,0.0,0,1,1


In [139]:
df = pd.read_csv('../datatest/bike-sharing.csv')
df['dteday'] = pd.to_datetime(df['dteday'])
df.set_index(df['dteday'], inplace=True)

df = df.resample('W').mean()
t = df[['cnt']]


In [140]:
size_train = int(len(t)*0.8)
train, test = t[0:size_train], t[size_train:]


Single exponential smoothing

In [164]:
from statsmodels.tsa.api import SimpleExpSmoothing

In [165]:
preds = []
tmp_train = train.copy()
for i in range(len(test)):
    mod = SimpleExpSmoothing(tmp_train)
    preds.append(mod.fit(smoothing_level=0.8, optimized=False).forecast(1).values[0])
    tmp_train = pd.concat([tmp_train, test[i:i+1]])
preds

C:\Users\Vladimir\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:427: FutureWarning:

After 0.13 initialization must be handled at model creation



[289.51025158126004,
 285.2115741257758,
 286.4756481584885,
 283.79989153645965,
 288.50759735491096,
 285.8919956614584,
 319.1879229418155,
 305.37091792169645,
 313.2598978700536,
 281.4519795740107,
 285.62849115289737,
 289.11141251629374,
 292.73180631278257,
 232.86151277770804,
 241.42260195673921,
 225.89880610563355,
 175.2511897925553,
 203.3711960423434,
 220.4932868275163,
 215.36532403216992,
 170.1968743302435,
 85.70202546845833]

In [166]:
trace1 = Scatter(y = t['cnt'], x= t.index, name = 'Orignial')
trace2 = Scatter(y = preds, x = test.index, name = 'Predicted')

iplot(Figure(data=[trace1, trace2]))

Double exponential smoothing

In [143]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [177]:
df['des_add']=ExponentialSmoothing(t, trend='add').fit().fittedvalues
df['des_mul']=ExponentialSmoothing(t, trend='mul').fit().fittedvalues


C:\Users\Vladimir\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:427: FutureWarning:

After 0.13 initialization must be handled at model creation

C:\Users\Vladimir\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:80: RuntimeWarning:

overflow encountered in matmul



In [178]:

iplot(Figure(data=[Scatter(x =df.index,
                           y = df['cnt'],
                           name = 'Original'),
                   Scatter(x = df.index,
                           y = df['des_add'],
                           name = 'Double Exponential Smoothing (Additive)'),
                   Scatter(x = df.index,
                           y = df['des_mul'],
                           name = 'Double Exponential Smoothing (Multiplicative)')]))

Triple exponential smoothing

In [180]:
df['tes_add']=ExponentialSmoothing(t, trend='add', seasonal='add',seasonal_periods=7).fit().fittedvalues
df['tes_mul']=ExponentialSmoothing(t, trend='mul', seasonal='mul', seasonal_periods=7).fit().fittedvalues

C:\Users\Vladimir\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:427: FutureWarning:

After 0.13 initialization must be handled at model creation

C:\Users\Vladimir\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:80: RuntimeWarning:

overflow encountered in matmul



In [181]:
iplot(Figure(data=[Scatter(x =df.index,
                           y = df['cnt'],
                           name = 'Original'),
                   Scatter(x = df.index,
                           y = df['tes_add'],
                           name = 'Tripple Exponential Smoothing (Additive)'),
                   Scatter(x = df.index,
                           y = df['tes_mul'],
                           name = 'Tripple Exponential Smoothing (Multiplicative)')]))